In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    # rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size,
               states, actions, next_states, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ ~s', Q = ~env(s, a)
    ################################################ ~s', Q = ~env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    for _ in range(11):
        next_states = tf.nn.tanh(next_states_logits)
        next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
        next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                       action_size=action_size, state_size=state_size, reuse=True)
        aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
        eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                        labels=tf.zeros_like(rates))) # min
        aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                         labels=tf.ones_like(rates))) # min
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.dones, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
# gamma = 0.99                   # future reward discount

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, 
              e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                #rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                #rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              #model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:0.7241 eloss:10.5700 aloss2:9.5478 exploreP:0.9983
Episode:1 meanR:18.0000 R:19.0000 rate:0.0380 aloss:0.7181 eloss:10.5675 aloss2:9.5508 exploreP:0.9964
Episode:2 meanR:16.6667 R:14.0000 rate:0.0280 aloss:0.7256 eloss:10.6239 aloss2:9.5057 exploreP:0.9951
Episode:3 meanR:15.5000 R:12.0000 rate:0.0240 aloss:0.7237 eloss:10.5025 aloss2:9.5874 exploreP:0.9939
Episode:4 meanR:17.0000 R:23.0000 rate:0.0460 aloss:0.7122 eloss:10.6269 aloss2:9.5057 exploreP:0.9916
Episode:5 meanR:19.5000 R:32.0000 rate:0.0640 aloss:0.7168 eloss:10.5480 aloss2:9.5530 exploreP:0.9885
Episode:6 meanR:21.7143 R:35.0000 rate:0.0700 aloss:0.7175 eloss:10.6006 aloss2:9.5034 exploreP:0.9851
Episode:7 meanR:21.1250 R:17.0000 rate:0.0340 aloss:0.7185 eloss:10.6728 aloss2:9.4477 exploreP:0.9834
Episode:8 meanR:19.8889 R:10.0000 rate:0.0200 aloss:0.7132 eloss:10.6381 aloss2:9.4751 exploreP:0.9824
Episode:9 meanR:19.2000 R:13.0000 rate:0.0260 aloss:0.7114 eloss:10.5744 

Episode:79 meanR:20.4625 R:26.0000 rate:0.0520 aloss:0.7035 eloss:5.0355 aloss2:18.9700 exploreP:0.8505
Episode:80 meanR:20.4568 R:20.0000 rate:0.0400 aloss:0.7069 eloss:4.9788 aloss2:19.1777 exploreP:0.8488
Episode:81 meanR:20.6098 R:33.0000 rate:0.0660 aloss:0.6963 eloss:4.8814 aloss2:19.4060 exploreP:0.8461
Episode:82 meanR:20.5060 R:12.0000 rate:0.0240 aloss:0.7042 eloss:4.8595 aloss2:19.5165 exploreP:0.8451
Episode:83 meanR:20.4524 R:16.0000 rate:0.0320 aloss:0.7320 eloss:4.8213 aloss2:19.6205 exploreP:0.8437
Episode:84 meanR:20.4941 R:24.0000 rate:0.0480 aloss:0.7411 eloss:4.7464 aloss2:19.8399 exploreP:0.8417
Episode:85 meanR:20.4535 R:17.0000 rate:0.0340 aloss:0.7145 eloss:4.6177 aloss2:20.2296 exploreP:0.8403
Episode:86 meanR:20.4828 R:23.0000 rate:0.0460 aloss:0.7150 eloss:4.5702 aloss2:20.4419 exploreP:0.8384
Episode:87 meanR:20.3864 R:12.0000 rate:0.0240 aloss:0.7169 eloss:4.5126 aloss2:20.6074 exploreP:0.8374
Episode:88 meanR:20.3708 R:19.0000 rate:0.0380 aloss:0.7046 elos

Episode:158 meanR:31.2700 R:16.0000 rate:0.0320 aloss:0.9088 eloss:1.6281 aloss2:54.7913 exploreP:0.6525
Episode:159 meanR:31.6100 R:45.0000 rate:0.0900 aloss:0.9360 eloss:1.6306 aloss2:55.4261 exploreP:0.6496
Episode:160 meanR:31.7200 R:30.0000 rate:0.0600 aloss:0.8897 eloss:1.6422 aloss2:56.0479 exploreP:0.6477
Episode:161 meanR:32.3600 R:82.0000 rate:0.1640 aloss:0.8727 eloss:1.6138 aloss2:56.8331 exploreP:0.6425
Episode:162 meanR:32.5500 R:37.0000 rate:0.0740 aloss:0.8780 eloss:1.6078 aloss2:57.2102 exploreP:0.6401
Episode:163 meanR:32.5000 R:12.0000 rate:0.0240 aloss:1.0458 eloss:1.5923 aloss2:57.7537 exploreP:0.6394
Episode:164 meanR:32.9100 R:56.0000 rate:0.1120 aloss:0.8533 eloss:1.6444 aloss2:57.9917 exploreP:0.6358
Episode:165 meanR:33.6000 R:84.0000 rate:0.1680 aloss:0.9249 eloss:1.5898 aloss2:59.3521 exploreP:0.6306
Episode:166 meanR:33.7000 R:19.0000 rate:0.0380 aloss:0.9331 eloss:1.5595 aloss2:60.0965 exploreP:0.6294
Episode:167 meanR:34.0600 R:55.0000 rate:0.1100 aloss:1

Episode:236 meanR:55.9100 R:73.0000 rate:0.1460 aloss:0.7532 eloss:1.5319 aloss2:79.7482 exploreP:0.4141
Episode:237 meanR:55.8600 R:25.0000 rate:0.0500 aloss:0.7458 eloss:1.5278 aloss2:77.2070 exploreP:0.4131
Episode:238 meanR:55.6300 R:44.0000 rate:0.0880 aloss:0.7323 eloss:1.5498 aloss2:74.9712 exploreP:0.4113
Episode:239 meanR:55.8700 R:40.0000 rate:0.0800 aloss:0.7381 eloss:1.5312 aloss2:73.6375 exploreP:0.4097
Episode:240 meanR:56.0000 R:27.0000 rate:0.0540 aloss:0.6959 eloss:1.5759 aloss2:72.7116 exploreP:0.4087
Episode:241 meanR:56.4700 R:60.0000 rate:0.1200 aloss:0.7124 eloss:1.5867 aloss2:72.1656 exploreP:0.4063
Episode:242 meanR:56.4500 R:53.0000 rate:0.1060 aloss:0.7302 eloss:1.5487 aloss2:72.6788 exploreP:0.4042
Episode:243 meanR:56.4600 R:33.0000 rate:0.0660 aloss:0.7016 eloss:1.4882 aloss2:74.6410 exploreP:0.4029
Episode:244 meanR:56.9000 R:110.0000 rate:0.2200 aloss:0.6973 eloss:1.5563 aloss2:75.6736 exploreP:0.3986
Episode:245 meanR:57.1800 R:68.0000 rate:0.1360 aloss:

Episode:314 meanR:34.1500 R:11.0000 rate:0.0220 aloss:0.7542 eloss:1.4797 aloss2:105.7838 exploreP:0.3426
Episode:315 meanR:33.5100 R:12.0000 rate:0.0240 aloss:0.7619 eloss:1.4332 aloss2:105.1449 exploreP:0.3422
Episode:316 meanR:32.8400 R:11.0000 rate:0.0220 aloss:0.7746 eloss:1.4301 aloss2:105.3509 exploreP:0.3418
Episode:317 meanR:32.0300 R:11.0000 rate:0.0220 aloss:0.7095 eloss:1.3995 aloss2:105.6118 exploreP:0.3415
Episode:318 meanR:31.6000 R:9.0000 rate:0.0180 aloss:0.7291 eloss:1.4218 aloss2:106.1879 exploreP:0.3412
Episode:319 meanR:30.8300 R:13.0000 rate:0.0260 aloss:0.7439 eloss:1.4371 aloss2:105.6040 exploreP:0.3407
Episode:320 meanR:30.5800 R:18.0000 rate:0.0360 aloss:0.7322 eloss:1.4987 aloss2:106.7300 exploreP:0.3401
Episode:321 meanR:30.3600 R:11.0000 rate:0.0220 aloss:0.7680 eloss:1.4325 aloss2:106.7129 exploreP:0.3398
Episode:322 meanR:29.9200 R:12.0000 rate:0.0240 aloss:0.7781 eloss:1.4510 aloss2:106.8702 exploreP:0.3394
Episode:323 meanR:29.3000 R:9.0000 rate:0.0180 

...

Episode:1093 meanR:18.6600 R:27.0000 rate:0.0540 aloss:0.9431 eloss:1.2955 aloss2:119.9851 exploreP:0.1415
Episode:1094 meanR:18.6300 R:12.0000 rate:0.0240 aloss:0.7868 eloss:1.2711 aloss2:118.8596 exploreP:0.1414
Episode:1095 meanR:18.7500 R:22.0000 rate:0.0440 aloss:0.7236 eloss:1.2579 aloss2:119.4430 exploreP:0.1411
Episode:1096 meanR:18.8000 R:20.0000 rate:0.0400 aloss:0.8676 eloss:1.2954 aloss2:118.9325 exploreP:0.1408
Episode:1097 meanR:18.8300 R:21.0000 rate:0.0420 aloss:0.7591 eloss:1.2883 aloss2:119.4601 exploreP:0.1405
Episode:1098 meanR:18.8900 R:21.0000 rate:0.0420 aloss:0.7752 eloss:1.2792 aloss2:119.5503 exploreP:0.1403
Episode:1099 meanR:18.9100 R:13.0000 rate:0.0260 aloss:0.7086 eloss:1.2575 aloss2:120.3429 exploreP:0.1401
Episode:1100 meanR:19.0500 R:27.0000 rate:0.0540 aloss:0.7740 eloss:1.2938 aloss2:120.1891 exploreP:0.1397
Episode:1101 meanR:19.0500 R:12.0000 rate:0.0240 aloss:0.8081 eloss:1.2947 aloss2:120.1930 exploreP:0.1396
Episode:1102 meanR:19.0000 R:14.0000 

Episode:1170 meanR:17.8800 R:11.0000 rate:0.0220 aloss:0.6891 eloss:1.2997 aloss2:118.4064 exploreP:0.1249
Episode:1171 meanR:17.7700 R:14.0000 rate:0.0280 aloss:0.7258 eloss:1.3206 aloss2:118.2268 exploreP:0.1247
Episode:1172 meanR:17.8100 R:18.0000 rate:0.0360 aloss:0.7289 eloss:1.2996 aloss2:118.4501 exploreP:0.1245
Episode:1173 meanR:17.7500 R:20.0000 rate:0.0400 aloss:0.7232 eloss:1.3318 aloss2:117.6476 exploreP:0.1243
Episode:1174 meanR:17.6900 R:11.0000 rate:0.0220 aloss:0.7860 eloss:1.2994 aloss2:118.1182 exploreP:0.1242
Episode:1175 meanR:17.6600 R:14.0000 rate:0.0280 aloss:0.6952 eloss:1.3113 aloss2:118.1163 exploreP:0.1240
Episode:1176 meanR:17.6700 R:20.0000 rate:0.0400 aloss:0.7127 eloss:1.3157 aloss2:117.8602 exploreP:0.1238
Episode:1177 meanR:17.6500 R:15.0000 rate:0.0300 aloss:0.7277 eloss:1.3194 aloss2:117.5418 exploreP:0.1236
Episode:1178 meanR:17.6600 R:14.0000 rate:0.0280 aloss:0.7477 eloss:1.3525 aloss2:117.2102 exploreP:0.1234
Episode:1179 meanR:17.5500 R:22.0000 

Episode:1247 meanR:30.5400 R:64.0000 rate:0.1280 aloss:0.6901 eloss:1.6206 aloss2:95.3480 exploreP:0.0983
Episode:1248 meanR:30.8500 R:65.0000 rate:0.1300 aloss:0.6868 eloss:1.5322 aloss2:98.3520 exploreP:0.0977
Episode:1249 meanR:31.1200 R:60.0000 rate:0.1200 aloss:0.6910 eloss:1.5387 aloss2:101.1048 exploreP:0.0972
Episode:1250 meanR:31.8600 R:86.0000 rate:0.1720 aloss:0.6897 eloss:1.5922 aloss2:103.3680 exploreP:0.0964
Episode:1251 meanR:32.3200 R:71.0000 rate:0.1420 aloss:0.6876 eloss:1.5124 aloss2:107.2461 exploreP:0.0958
Episode:1252 meanR:32.5700 R:37.0000 rate:0.0740 aloss:0.6870 eloss:1.5715 aloss2:109.3483 exploreP:0.0955
Episode:1253 meanR:32.7700 R:42.0000 rate:0.0840 aloss:0.6959 eloss:1.5409 aloss2:111.4856 exploreP:0.0951
Episode:1254 meanR:33.1500 R:59.0000 rate:0.1180 aloss:0.6828 eloss:1.5120 aloss2:114.4919 exploreP:0.0946
Episode:1255 meanR:33.7700 R:76.0000 rate:0.1520 aloss:0.6997 eloss:1.4564 aloss2:118.0111 exploreP:0.0940
Episode:1256 meanR:34.1800 R:58.0000 ra

Episode:1324 meanR:36.7600 R:10.0000 rate:0.0200 aloss:0.7240 eloss:1.2653 aloss2:184.5642 exploreP:0.0789
Episode:1325 meanR:36.3200 R:8.0000 rate:0.0160 aloss:0.7245 eloss:1.3548 aloss2:185.0153 exploreP:0.0788
Episode:1326 meanR:35.8100 R:9.0000 rate:0.0180 aloss:0.7497 eloss:1.8729 aloss2:181.6067 exploreP:0.0788
Episode:1327 meanR:35.4800 R:9.0000 rate:0.0180 aloss:0.7848 eloss:1.7820 aloss2:179.1108 exploreP:0.0787
Episode:1328 meanR:34.6100 R:10.0000 rate:0.0200 aloss:0.7043 eloss:1.2648 aloss2:181.6056 exploreP:0.0786
Episode:1329 meanR:33.9800 R:8.0000 rate:0.0160 aloss:0.7136 eloss:1.2806 aloss2:180.6895 exploreP:0.0786
Episode:1330 meanR:33.5200 R:13.0000 rate:0.0260 aloss:0.7288 eloss:1.2400 aloss2:181.4981 exploreP:0.0785
Episode:1331 meanR:32.8000 R:10.0000 rate:0.0200 aloss:0.7581 eloss:1.2855 aloss2:183.4868 exploreP:0.0784
Episode:1332 meanR:32.4400 R:9.0000 rate:0.0180 aloss:0.7475 eloss:1.4396 aloss2:183.0451 exploreP:0.0784
Episode:1333 meanR:32.1700 R:10.0000 rate:

Episode:1401 meanR:9.7900 R:11.0000 rate:0.0220 aloss:0.8266 eloss:1.2511 aloss2:188.6264 exploreP:0.0739
Episode:1402 meanR:9.7800 R:9.0000 rate:0.0180 aloss:0.8919 eloss:1.2885 aloss2:189.4176 exploreP:0.0739
Episode:1403 meanR:9.7800 R:10.0000 rate:0.0200 aloss:0.8488 eloss:1.3699 aloss2:187.6335 exploreP:0.0738
Episode:1404 meanR:9.7700 R:10.0000 rate:0.0200 aloss:0.8517 eloss:1.2776 aloss2:187.7215 exploreP:0.0738
Episode:1405 meanR:9.7600 R:10.0000 rate:0.0200 aloss:0.8582 eloss:1.2891 aloss2:186.2629 exploreP:0.0737
Episode:1406 meanR:9.7800 R:12.0000 rate:0.0240 aloss:0.9021 eloss:1.2601 aloss2:187.7152 exploreP:0.0736
Episode:1407 meanR:9.7800 R:9.0000 rate:0.0180 aloss:0.9118 eloss:1.3375 aloss2:187.5836 exploreP:0.0736
Episode:1408 meanR:9.8300 R:13.0000 rate:0.0260 aloss:0.8772 eloss:1.3060 aloss2:186.8240 exploreP:0.0735
Episode:1409 meanR:9.8200 R:10.0000 rate:0.0200 aloss:0.8801 eloss:1.3811 aloss2:185.6925 exploreP:0.0734
Episode:1410 meanR:9.7500 R:10.0000 rate:0.0200 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [39]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
